In [108]:
from rdkit import Chem
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')   
RDLogger.DisableLog('rdkit.*')

In [111]:
dfDataset = pd.read_csv("data/datasetCSV/GeneralDataset.csv")
dfDataset

,dataset,smiles,target
0,ADRA1A,COC(=O)C1=C(C)NC(=O)N(C(=O)NCCC[NH+]2CCN(c3ccc...,-4.605170
1,ADRA1A,COc1cc2[nH+]c(N3CCN(C(=O)c4ccco4)CC3)nc(N)c2cc1OC,-4.605170
2,ADRA1A,COC(=O)C1=C(C)NC(=O)N(C(=O)NCCC[NH+]2CCN(c3ccc...,-3.912023
3,ADRA1A,CC(C)Oc1ccccc1N1CCN(Cc2cccc(C(=O)N3CCCCC3)c2)CC1,-3.912023
4,ADRA1A,CCOc1ccccc1OCC[NH2+][C@H](C)Cc1ccc(OC)c(S(N)(=...,-3.540459
...,...,...,...
26531,SOL,Cc1cc(C)cc(C(=O)NCCCNc2ncccn2)c1,1.720449
26532,SOL,CCc1noc(COc2c(C)ccnc2Cl)n1,1.366983
26533,SOL,CC(C)(C)Cc1nnc(-c2cnc3onc(C4CCCC4)c3c2)o1,0.600973
26534,SOL,Cc1nonc1C(=O)NCCc1c[nH]c2cccc(C3(O)CCOCC3)c12,1.786751


In [119]:
nameDataset = ["ADRA1A", "ALOX5AP", "ATR", "DPP4", "HLMC", "JAK1", "JAK2", "KOR", "LIPO", "MUSC1", "MUSC2", "SOL"]
dfs=[]
for name in nameDataset:
    dfF = pd.read_pickle(f"data/features/{name}/{name}_ECFP4.pkl")
    dfF = dfF.drop_duplicates(subset=['smiles'], keep='first')
    dfF = dfF.drop("target", axis=1)
    dfF["dataset"]=name
    dfs.append(dfF)

dfFTotal = pd.concat(dfs,ignore_index=True)
dfFTotal



,0,1,2,3,4,5,6,7,8,9,...,1016,1017,1018,1019,1020,1021,1022,1023,smiles,dataset
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,COC(=O)C1=C(C)NC(=O)N(C(=O)NCCC[NH+]2CCN(c3ccc...,ADRA1A
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,COc1cc2[nH+]c(N3CCN(C(=O)c4ccco4)CC3)nc(N)c2cc1OC,ADRA1A
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,COC(=O)C1=C(C)NC(=O)N(C(=O)NCCC[NH+]2CCN(c3ccc...,ADRA1A
3,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CC(C)Oc1ccccc1N1CCN(Cc2cccc(C(=O)N3CCCCC3)c2)CC1,ADRA1A
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CCOc1ccccc1OCC[NH2+][C@H](C)Cc1ccc(OC)c(S(N)(=...,ADRA1A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26530,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,Cc1cc(C)cc(C(=O)NCCCNc2ncccn2)c1,SOL
26531,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CCc1noc(COc2c(C)ccnc2Cl)n1,SOL
26532,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,CC(C)(C)Cc1nnc(-c2cnc3onc(C4CCCC4)c3c2)o1,SOL
26533,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Cc1nonc1C(=O)NCCc1c[nH]c2cccc(C3(O)CCOCC3)c12,SOL


In [120]:
df = pd.merge(dfDataset, dfFTotal, how="left", on=["smiles","dataset"])
df

,dataset,smiles,target,0,1,2,3,4,5,6,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,ADRA1A,COC(=O)C1=C(C)NC(=O)N(C(=O)NCCC[NH+]2CCN(c3ccc...,-4.605170,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,ADRA1A,COc1cc2[nH+]c(N3CCN(C(=O)c4ccco4)CC3)nc(N)c2cc1OC,-4.605170,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,ADRA1A,COC(=O)C1=C(C)NC(=O)N(C(=O)NCCC[NH+]2CCN(c3ccc...,-3.912023,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
3,ADRA1A,CC(C)Oc1ccccc1N1CCN(Cc2cccc(C(=O)N3CCCCC3)c2)CC1,-3.912023,0.0,1.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,ADRA1A,CCOc1ccccc1OCC[NH2+][C@H](C)Cc1ccc(OC)c(S(N)(=...,-3.540459,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26531,SOL,Cc1cc(C)cc(C(=O)NCCCNc2ncccn2)c1,1.720449,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
26532,SOL,CCc1noc(COc2c(C)ccnc2Cl)n1,1.366983,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
26533,SOL,CC(C)(C)Cc1nnc(-c2cnc3onc(C4CCCC4)c3c2)o1,0.600973,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
26534,SOL,Cc1nonc1C(=O)NCCc1c[nH]c2cccc(C3(O)CCOCC3)c12,1.786751,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [129]:
print(df.isna().sum().sum())
df.dropna(ignore_index=True)

15360


,dataset,smiles,target,0,1,2,3,4,5,6,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,ADRA1A,COC(=O)C1=C(C)NC(=O)N(C(=O)NCCC[NH+]2CCN(c3ccc...,-4.605170,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,ADRA1A,COc1cc2[nH+]c(N3CCN(C(=O)c4ccco4)CC3)nc(N)c2cc1OC,-4.605170,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,ADRA1A,COC(=O)C1=C(C)NC(=O)N(C(=O)NCCC[NH+]2CCN(c3ccc...,-3.912023,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
3,ADRA1A,CC(C)Oc1ccccc1N1CCN(Cc2cccc(C(=O)N3CCCCC3)c2)CC1,-3.912023,0.0,1.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,ADRA1A,CCOc1ccccc1OCC[NH2+][C@H](C)Cc1ccc(OC)c(S(N)(=...,-3.540459,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26516,SOL,Cc1cc(C)cc(C(=O)NCCCNc2ncccn2)c1,1.720449,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
26517,SOL,CCc1noc(COc2c(C)ccnc2Cl)n1,1.366983,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
26518,SOL,CC(C)(C)Cc1nnc(-c2cnc3onc(C4CCCC4)c3c2)o1,0.600973,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
26519,SOL,Cc1nonc1C(=O)NCCc1c[nH]c2cccc(C3(O)CCOCC3)c12,1.786751,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
